In [3]:
# ! git clone https://hf-mirror.com/datasets/openai/gsm8k

In [4]:
import pandas as pd
gsm8k_train = pd.read_parquet('gsm8k/main/train-00000-of-00001.parquet', engine='pyarrow')
gsm8k_train

,question,answer
0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<..."
3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....
4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...
...,...,...
7468,"Very early this morning, Elise left home in a ...","For the distance she traveled, Elise paid 23 -..."
7469,Josh is saving up for a box of cookies. To rai...,He makes $.5 profit on each bracelet because 1...
7470,Colin can skip at six times the speed that Bra...,Tony can skip at twice the speed that Bruce ca...
7471,"Janet, a third grade teacher, is picking up th...",Janet needs 35 lunches for the kids + 5 for th...


### Let's print random 20 questions and choose three of them

In [5]:
# choose the following IDs: 1297, 4336, and 5196
pd.set_option('display.max_colwidth', None)
sample_20 = gsm8k_train.sample(20, random_state=42)
sample_20

,question,answer
1297,"In Professor Plum's biology class there are 40 students. Of those students, 80 percent have puppies. Of those who have puppies, 25% also have parrots. How many students have both puppies and parrots?","We start with the initial numbers of students, 40 and multiply that by .8 for 40 * 0.8 = <<40*0.8=32>>32 who own puppies.\nThat the number of students with puppies, 32, and multiply that by .25 to find out how many own both puppies and parrots, 32 * 0.25 = <<32*0.25=8>>8 who own puppies and parrots.\nThe answer is <<8=8>>8.\n#### 8"
576,"Diane bought twenty more apples than Cecile. If Cecile bought 15 apples, how many apples did they buy altogether?","Diane bought 15 + 20 = <<15+20=35>>35 apples.\nTherefore, they bought 15 + 35 = <<15+35=50>>50 apples altogether.\n#### 50"
5462,"Ann can skate 6 miles an hour. Her friend Glenda can skate 8 miles an hour. If they start in the same place and skate in straight lines in opposite directions for 3 hours, how many miles apart do they end up?",First find how far Glenda goes in 3 hours by multiplying her speed by the number of hours she travels: 3 hours * 8 miles/hour = <<3*8=24>>24 miles\nThen do the same thing for Ann: 3 hours * 6 miles/hour = <<3*6=18>>18 miles\nNow add the number of miles both people skated to find the total distance between them: 18 miles + 24 miles = <<18+24=42>>42 miles\n#### 42
4336,"Running for 2 hours, Jonah burnt 30 calories every hour. How many more calories would he have lost if he would have run for five hours?","When Jonah ran for 2 hours, burning 30 calories every hour, he burnt a total of 2*30=<<2*30=60>>60 calories.\nIf he had run for five hours, losing 30 calories every hour, Jonah would have burnt 5*30=<<5*30=150>>150 calories.\nThe difference in the number of calories Jonah would have burnt if he ran for five hours instead of 2 hours is 150-60=<<150-60=90>>90 calories.\n#### 90"
7105,"The city of Richmond has 1000 more people than Victoria. Victoria has 4 times as many people as Beacon. If Richmond has 3000 people, how many people are there in Beacon?",Victoria has 3000-1000=<<3000-1000=2000>>2000 people.\nBeacon has 2000/4=<<2000/4=500>>500 people.\n#### 500
6808,"To get his fill of oysters, Crabby has to eat at least twice as many oysters as Squido does. If Squido eats 200 oysters, how many oysters do they eat altogether?","If Squido eats 200 oysters, when Crabby eats twice as many oysters as Squido does, he eats 2*200 = 400 oysters.\nTogether, they eat 400+200 = <<400+200=600>>600 oysters.\n#### 600"
4404,John sells 20 woodburning for $15 each. The wood cost $100. How much does he make in profit?,He sells the woodburning for 20*15=$<<20*15=300>>300\nSo he makes a profit of 300-100=$<<300-100=200>>200\n#### 200
1649,"In a field of 500 clovers, 20% have four leaves and one quarter of these are purple clovers. Assuming these proportions are exactly correct, how many clovers in the field are both purple and four-leaved?",There are 500/5= <<500/5=100>>100 four leaf clovers\nThere are 100/4= <<100/4=25>>25 purple four leaf clovers\n#### 25
5958,"Tony lifts weights as a form of exercise. He can lift 90 pounds with one arm in the exercise known as ""the curl."" In an exercise known as ""the military press,"" he can lift over his head twice the weight that he can curl. His favorite exercise is known as ""the squat"" and he can squat 5 times the weight that he can lift in the military press. How much weight, in pounds, can Tony lift in the squat exercise?","If Tony can curl 90 pounds, he can military press 2*90=<<90*2=180>>180 pounds.\nTherefore, Tony can squat 5*180=<<5*180=900>>900 pounds.\n#### 900"
6366,"Elsa started the day with 40 marbles. At breakfast, she lost 3 marbles while playing. At lunchtime, she gave her best friend Susie 5 marbles. In the afternoon, Elsa's mom bought her a new bag with 12 marbles. Susie came back and gave Elsa twice as many marbles as she received at lunch. How many marbles did Elsa end the day with?",E

In [6]:
chosen_ids = [1297, 4336, 5196]
chosen_questions = gsm8k_train[gsm8k_train.index.isin(chosen_ids)]
chosen_questions

,question,answer
1297,"In Professor Plum's biology class there are 40 students. Of those students, 80 percent have puppies. Of those who have puppies, 25% also have parrots. How many students have both puppies and parrots?","We start with the initial numbers of students, 40 and multiply that by .8 for 40 * 0.8 = <<40*0.8=32>>32 who own puppies.\nThat the number of students with puppies, 32, and multiply that by .25 to find out how many own both puppies and parrots, 32 * 0.25 = <<32*0.25=8>>8 who own puppies and parrots.\nThe answer is <<8=8>>8.\n#### 8"
4336,"Running for 2 hours, Jonah burnt 30 calories every hour. How many more calories would he have lost if he would have run for five hours?","When Jonah ran for 2 hours, burning 30 calories every hour, he burnt a total of 2*30=<<2*30=60>>60 calories.\nIf he had run for five hours, losing 30 calories every hour, Jonah would have burnt 5*30=<<5*30=150>>150 calories.\nThe difference in the number of calories Jonah would have burnt if he ran for five hours instead of 2 hours is 150-60=<<150-60=90>>90 calories.\n#### 90"
5196,"Mark is reading books, for 2 hours each day. He decided to increase his time spent on reading books weekly, by 4 hours. How much time does Mark want to spend during one week on reading books?","Currently, Mark is reading books for 2 * 7 = <<2*7=14>>14 hours weekly.\nHis goal is to read books, for 14 + 4 = <<14+4=18>>18 hours during one week.\n#### 18"


In [7]:
few_shot_examples_ids = [5833, 5368, 1669, 3463, 1055, 1025]
df_for_few_shot = gsm8k_train[gsm8k_train.index.isin(few_shot_examples_ids)]
df_for_few_shot['difficulty'] = [2, 2, 2, 2, 3, 2] # add difficulty levels based on how many steps of computation are needed
df_for_few_shot[['reasoning', 'answer']] = df_for_few_shot['answer'].str.split('\n####', expand=True)
df_for_few_shot

/var/folders/d3/77342y0n20qgyxgcd7fxq_ww0000gn/T/ipykernel_77998/4254509031.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_few_shot['difficulty'] = [2, 2, 2, 2, 3, 2] # add difficulty levels based on how many steps of computation are needed
/var/folders/d3/77342y0n20qgyxgcd7fxq_ww0000gn/T/ipykernel_77998/4254509031.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_few_shot[['reasoning', 'answer']] = df_for_few_shot['answer'].str.split('\n####', expand=True)


,question,answer,difficulty,reasoning
1025,A survey conducted by the school showed that only 20% of the 800 parents agree to a tuition fee increase. How many parents disagree with the tuition fee increase?,640,2,"There were 800 x 20/100 = <<800*20/100=160>>160 parents who agreed to the tuition fee increase.\nSo, there were 800 - 160 = <<800-160=640>>640 parents who disagreed."
1055,"New York is two-thirds as populated as all of New England. If New England has 2100000 people, calculate the combined population of both states.",3500000,2,"New York is two-thirds as populated as all of New England, meaning there are 2/3*2100000 = <<2/3*2100000=1400000>>1400000 people living in New York.\nThe combined population of both states is 1400000+2100000 = <<1400000+2100000=3500000>>3500000"
1669,"In a field where there are 200 animals, there are 40 cows, 56 sheep and goats. How many goats are there?",104,2,"We have 56 + 40 = <<56+40=96>>96 animals other than goats.\nSo, we have 200 – 96 = <<200-96=104>>104 goats"
3463,John buys thank you cards. He decides to buy them for people who got him Christmas and birthday presents. He sent 20 for Christmas gifts and 15 for birthday gifts. If each card cost $2 how much did he spend on cards?,70,2,He bought 20+15=<<20+15=35>>35 cards\nSo he spent 35*2=$<<35*2=70>>70
5368,"Lisa is making a pizza. She uses 30 pieces of pepperoni for a pizza, twice as many pieces of ham, and 12 more pieces of sausage than pepperoni. If there are 6 slices of pizza, and everything was distributed evenly, how many pieces of meat altogether are on each slice?",22,3,There are 30*2 = <<30*2=60>>60 pieces of ham put on the pizza.\nThere are 30+12 = <<30+12=42>>42 pieces of sausage put on the pizza.\nThere are 30/6 = <<30/6=5>>5 pieces of pepperoni on each pizza slice.\nThere are 60/6 = <<60/6=10>>10 pieces of ham on each pizza slice.\nThere are 42/6 = <<42/6=7>>7 pieces of sausage on each pizza slice.\nThere are 5+10+7 = <<5+10+7=22>>22 pieces altogether on each slice of pizza.
5833,"In a shop, there is a sale of clothes. Every shirt costs $5, every hat $4, and a pair of jeans $10. How much do you need to pay for three shirts, two pairs of jeans, and four hats?",51,2,"Three shirts cost 3 * 5 = $<<3*5=15>>15.\nTwo pairs of jeans would cost 2 * 10 = $<<2*10=20>>20.\nFour hats would in total cost 4 * 4 = $<<4*4=16>>16.\nSo for three shirts, two pairs of jeans, and four hats, you would need to pay 15 + 20 + 16 = $<<15+20+16=51>>51."


### Construct the json for few-shot learning example

+ In our case, the number of examples will be 2 (json files), each having three questions and answers

In [8]:
import json

# Split the dataframe into two parts
first_half = df_for_few_shot.iloc[:3]
second_half = df_for_few_shot.iloc[3:]

# Convert the first half to a dictionary
first_half_dict = {
    "Question ID": first_half.index.tolist(),
    "Reasoning Process": first_half['reasoning'].tolist(),
    "Final Answer": first_half['answer'].tolist(),
    "Difficulty Classification": first_half['difficulty'].tolist()
}

# Convert the second half to a dictionary
second_half_dict = {
    "Question ID": second_half.index.tolist(),
    "Reasoning Process": second_half['reasoning'].tolist(),
    "Final Answer": second_half['answer'].tolist(),
    "Difficulty Classification": second_half['difficulty'].tolist()
}

# Save the first half to a JSON file
with open('first_half.json', 'w') as f:
    json.dump(first_half_dict, f, indent=4)

# Save the second half to a JSON file
with open('second_half.json', 'w') as f:
    json.dump(second_half_dict, f, indent=4)

print("JSON files created successfully!")

JSON files created successfully!


In [9]:
df_for_few_shot.index.tolist()

[1025, 1055, 1669, 3463, 5368, 5833]

### Analysis

In [13]:
chosen_questions

,question,answer
1297,"In Professor Plum's biology class there are 40 students. Of those students, 80 percent have puppies. Of those who have puppies, 25% also have parrots. How many students have both puppies and parrots?","We start with the initial numbers of students, 40 and multiply that by .8 for 40 * 0.8 = <<40*0.8=32>>32 who own puppies.\nThat the number of students with puppies, 32, and multiply that by .25 to find out how many own both puppies and parrots, 32 * 0.25 = <<32*0.25=8>>8 who own puppies and parrots.\nThe answer is <<8=8>>8.\n#### 8"
4336,"Running for 2 hours, Jonah burnt 30 calories every hour. How many more calories would he have lost if he would have run for five hours?","When Jonah ran for 2 hours, burning 30 calories every hour, he burnt a total of 2*30=<<2*30=60>>60 calories.\nIf he had run for five hours, losing 30 calories every hour, Jonah would have burnt 5*30=<<5*30=150>>150 calories.\nThe difference in the number of calories Jonah would have burnt if he ran for five hours instead of 2 hours is 150-60=<<150-60=90>>90 calories.\n#### 90"
5196,"Mark is reading books, for 2 hours each day. He decided to increase his time spent on reading books weekly, by 4 hours. How much time does Mark want to spend during one week on reading books?","Currently, Mark is reading books for 2 * 7 = <<2*7=14>>14 hours weekly.\nHis goal is to read books, for 14 + 4 = <<14+4=18>>18 hours during one week.\n#### 18"


##### Zero-shot learning

In [11]:
# Load the JSON file
with open('gpt-4_zero_shot.json', 'r') as f:
    data = json.load(f)

# Convert the JSON data to a pandas DataFrame
zero_shot_results = pd.DataFrame({
    'question': data['Question ID'],
    'reasoning': data['Reasoning Process'],
    'answer': data['Final Answer'],
    'difficulty': data['Difficulty Classification']
})

zero_shot_results

,question,reasoning,answer,difficulty
0,1297,"There are 40 students; 80% have puppies, so 0.80×40 = 32 students. Of those, 25% also have parrots, so 0.25×32 = 8 students.",8,1
1,4336,Jonah burns 30 calories per hour. In 2 hours he burns 2×30 = 60 calories; in 5 hours he would burn 5×30 = 150 calories. The difference is 150 − 60 = 90 calories.,90,1
2,5196,"Mark reads 2 hours per day for 7 days, so 2×7 = 14 hours per week. He adds 4 hours weekly, giving 14 + 4 = 18 hours.",18,1


**The results are good, because the questions are very simple. However, the estimation of difficulty is different from the way I was defining it. Let's see if it changes with the few-shot learning.**

##### Few-shot learning

In [14]:
# Load the JSON file
with open('gpt-4_few_shots.json', 'r') as f:
    data = json.load(f)

# Convert the JSON data to a pandas DataFrame
few_shots_results = pd.DataFrame({
    'question': data['Question ID'],
    'reasoning': data['Reasoning Process'],
    'answer': data['Final Answer'],
    'difficulty': data['Difficulty Classification']
})

few_shots_results

,question,reasoning,answer,difficulty
0,1297,"There are 40 students total, and 80% have puppies: 40 × 0.80 = <<40*0.80=32>>32. Of those 32, 25% also have parrots: 32 × 0.25 = <<32*0.25=8>>8.",8,1
1,4336,"Jonah burns 30 calories per hour, so in 2 hours he burns 30 × 2 = <<30*2=60>>60 calories. In 5 hours he would burn 30 × 5 = <<30*5=150>>150 calories. The extra calories lost are 150 − 60 = <<150-60=90>>90.",90,1
2,5196,"Mark reads 2 hours per day for 7 days: 2 × 7 = <<2*7=14>>14 hours per week. He increases this by 4 hours, giving 14 + 4 = <<14+4=18>>18 hours.",18,1


**The content did not change much. Meanwhile, the answer looks more like the examples in the dataset (mainly due to the calculations in the <<>> brackets). Hence, few-shot learning worked, but not sure if it was needed for such simple problems, given that modern reasoning models are already powerful enough.**